# Tarea 1 Máquinas de Aprendizaje

## Ejercicio 1

a) Construcción del dataframe. La primera columna del dataframe original es redundante para la indexación, mientras que la columna $Train$ nos permite identificar cuales ejemplos serán parte del training set ($Train = T$) y del testing set ($Train = F$). Para explicitar qué ejemplos son del testing set se invierten los valores de verdad de dicha columna. Finalmente la columna ya utilizada se descarta para quedarnos con las columnas de predictores.

In [3]:
import numpy as np
import pandas as pd

url = 'http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data'
df = pd.read_csv(url, sep='\t', header=0)
# Remover columna con indices redundantes
df = df.drop('Unnamed: 0', axis=1)
# Obtener columna con la etiqueta Train y reemplazar valores booleanos. Estos ejemplos seran de entrenamiento
istrain_str = df['train']
istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
# Listar como testing el resto de valores falsos de la columna anterior
istest = np.logical_not(istrain)
# Una vez procesado los datos, eliminar la columna train para almacenar los predictores relevantes
df = df.drop('train', axis=1)

b) Descripción del dataset. El dataset posee 9 atributos y 97 samples con valores enteros y reales.

In [4]:
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Data columns (total 9 columns):
lcavol     97 non-null float64
lweight    97 non-null float64
age        97 non-null int64
lbph       97 non-null float64
svi        97 non-null int64
lcp        97 non-null float64
gleason    97 non-null int64
pgg45      97 non-null int64
lpsa       97 non-null float64
dtypes: float64(5), int64(4)
memory usage: 7.6 KB


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,1.350010,3.628943,63.865979,0.100356,0.216495,-0.179366,6.752577,24.381443,2.478387
std,1.178625,0.428411,7.445117,1.450807,0.413995,1.398250,0.722134,28.204035,1.154329
min,-1.347074,2.374906,41.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,-0.430783
25%,0.512824,3.375880,60.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,1.731656
50%,1.446919,3.623007,65.000000,0.300105,0.000000,-0.798508,7.000000,15.000000,2.591516
75%,2.127041,3.876396,68.000000,1.558145,0.000000,1.178655,7.000000,40.000000,3.056357
max,3.821004,4.780383,79.000000,2.326302,1.000000,2.904165,9.000000,100.000000,5.582932


c) Normalización de datos. Este preprocesamiento de los datos es importante pues las features originales pueden venir en distintas escalas por lo tanto nuestro algoritmo de aprendizaje no funcionará correctamente, por ejemplo al utilizar funciones objetivo que incluyan métricas, los datos con mayor rango tenderán a dominar sobre los de menor rango; por otra parte también podría darse el caso de que la convergencia en algoritmos que usen gradiente descendiente sea lenta o imprecisa.

In [5]:
from sklearn.preprocessing import StandardScaler
# Por defecto centra y escala la data.
scaler = StandardScaler(with_mean=True, with_std=True)
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
#  Deseamos aprender a predecir el feature lpsa, por lo que la recuperamos del original
df_scaled['lpsa'] = df['lpsa']

d) Regresión lineal con Mínimos Cuadrados. En primer lugar extraemos la última columna de los datos, que corresponde al output $y$ de cada ejemplo. La nueva columna añadida corresponde al bias clásico...
El argumento pasado al constructor de LinearRegression indica que no se calculará intercepto para el modelo (ya lo hemos hecho a través de normalizar e ingresar la columna con bias 1)

In [6]:
import sklearn.linear_model as lm
X = df_scaled.ix[:,:-1]
# Agregamos la columna de bias con 1
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
# Obtener los datos de output conocidos y extraer test & training set
y = df_scaled['lpsa']
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[istest]
ytest = y[istest]
linreg = lm.LinearRegression(fit_intercept=False)
linreg.fit(Xtrain, ytrain)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

e) Tabla de pesos (coeficientes) y Z-scores para cada variable. Los Z-scores miden el efecto de descartar alguna variable del modelo. Bajo la hipótesis nula de que algún coeficiente debe ser 0, su Z-score entonces se distribuye como una distribución $t_{N-p-1}$. Buscamos en la tabla y vemos que aproximadamente valores superiores a 2 en mayor absoluto

Cuando deseamos un 5% de significancia, las variables que resultarán más importantes deberán tener Z-score en valor abosluto mayor que 2, lo que indica una significancia del 5%. Las variables mas importante spor lo tanto son en orden de Z-score decreciente **lcavol**, **svi**, **lcp** y **pgg45**

In [12]:
# Correlacion entre variables
# print Xtrain.drop('intercept', axis=1).corr()

# Tabla con coefficientes y sus Z-score
coeffs = linreg.coef_
Table = pd.DataFrame(coeffs, index=X.columns, columns=['Coefficent'])
Table['Std. Error'] = Xtrain.std() / np.sqrt(Xtrain.shape[0])
Table['Z-score'] = Table['Coefficent'].div(Table['Std. Error'], axis=0)
Table

,Coefficent,Std. Error,Z-score
lcavol,0.676016,0.129469,5.221460
lweight,0.261694,0.136618,1.915519
age,-0.140734,0.123746,-1.137281
lbph,0.209061,0.123892,1.687447
svi,0.303623,0.124582,2.437133
lcp,-0.287002,0.123022,-2.332923
gleason,-0.021195,0.120547,-0.175822
pgg45,0.265576,0.127584,2.081583
intercept,2.464933,0.000000,inf
